In [1]:
import numpy as np
np.__version__
import math

In [2]:
class TicTacToe:
    def __init__(self):
        self.row_count = 3
        self.column_count = 3
        self.action_size = self.row_count * self.column_count
        
    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))
    
    def get_next_state(self, state, action, player):
        row = action // self.column_count
        column = action % self.column_count
        state[row, column] = player
        return state
    
    def get_valid_moves(self, state):
        return (state.reshape(-1) == 0).astype(np.uint8)
    
    def check_win(self, state, action):
        if action == None:
            return False
        
        row = action // self.column_count
        column = action % self.column_count
        player = state[row, column]
        
        return (
            np.sum(state[row, :]) == player * self.column_count
            or np.sum(state[:, column]) == player * self.row_count
            or np.sum(np.diag(state)) == player * self.row_count
            or np.sum(np.diag(np.flip(state, axis=0))) == player * self.row_count
        )
    
    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False
    
    def get_opponent(self, player):
        return -player
    
    def get_opponent_value(self, value):
        return -value
    
    def change_perspective(self, state, player):
        return state * player

In [3]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        
        self.children = []
        self.expandable_moves = game.get_valid_moves(state)
        
        self.value_sum = 0
        self.visit_count = 0
        
    def is_fully_expanded(self):
        return np.sum(self.expandable_moves) == 0 and len(self.children) > 0
    
    def select(self):
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
                
        return best_child
    
    def get_ucb(self, child):
        q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count)
    
    def expand(self):
        action = np.random.choice(np.where(self.expandable_moves == 1)[0])
        self.expandable_moves[action] = 0
        
        child_state = self.state.copy()
        child_state = self.game.get_next_state(child_state, action, 1)
        child_state = self.game.change_perspective(child_state, player=-1)
        
        child = Node(self.game, self.args, child_state, self, action)
        self.children.append(child)
        return child
    
    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        value = self.game.get_opponent_value(value)
        
        if is_terminal:
            return value
        
        rollout_state = self.state.copy()
        rollout_player = 1
        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(np.where(valid_moves == 1)[0])
            rollout_state = self.game.get_next_state(rollout_state, action, rollout_player)
            value, is_terminal = self.game.get_value_and_terminated(rollout_state, action)
            if is_terminal:
                if rollout_player == -1:
                    value = self.game.get_opponent_value(value)
                return value    
            
            rollout_player = self.game.get_opponent(rollout_player)
            
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args
        
    def search(self, state):
        root = Node(self.game, self.args, state)
        
        for search in range(self.args['num_searches']):
            node = root
            
            while node.is_fully_expanded():
                node = node.select()
                
            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = self.game.get_opponent_value(value)
            
            if not is_terminal:
                node = node.expand()
                value = node.simulate()
                
            node.backpropagate(value)    
            
            
        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs
        
        
        

In [4]:
tictactoe = TicTacToe()
player = 1

args = {
    'C': 1.41,
    'num_searches': 1000
}

mcts = MCTS(tictactoe, args)

state = tictactoe.get_initial_state()


while True:
    print(state)
    
    if player == 1:
        valid_moves = tictactoe.get_valid_moves(state)
        print("valid_moves", [i for i in range(tictactoe.action_size) if valid_moves[i] == 1])
        action = int(input(f"{player}:"))

        if valid_moves[action] == 0:
            print("action not valid")
            continue
            
    else:
        neutral_state = tictactoe.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)
        
    state = tictactoe.get_next_state(state, action, player)
    
    value, is_terminal = tictactoe.get_value_and_terminated(state, action)
    
    if is_terminal:
        print(state)
        if value == 1:
            print(player, "won")
        else:
            print("draw")
        break
        
    player = tictactoe.get_opponent(player)



[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid_moves [0, 1, 2, 3, 4, 5, 6, 7, 8]
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
valid_moves [0, 2, 3, 5, 6, 7, 8]
[[ 1.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
[[ 1.  1. -1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
valid_moves [3, 5, 6, 7, 8]
action not valid
[[ 1.  1. -1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
valid_moves [3, 5, 6, 7, 8]
[[ 1.  1. -1.]
 [ 0. -1.  0.]
 [ 1.  0.  0.]]
[[ 1.  1. -1.]
 [-1. -1.  0.]
 [ 1.  0.  0.]]
valid_moves [5, 7, 8]
[[ 1.  1. -1.]
 [-1. -1.  1.]
 [ 1.  0.  0.]]
[[ 1.  1. -1.]
 [-1. -1.  1.]
 [ 1. -1.  0.]]
valid_moves [8]
[[ 1.  1. -1.]
 [-1. -1.  1.]
 [ 1. -1.  1.]]
draw


In [6]:
import chess
import chess.svg
import math
import pygame
from pygame.locals import *

# Unicode symbols for chess pieces
UNICODE_PIECES = {
    'K': '♔', 'Q': '♕', 'R': '♖', 'N': '♘', 'B': '♗', 'P': '♙',
    'k': '♚', 'q': '♛', 'r': '♜', 'n': '♞', 'b': '♝', 'p': '♟',
}

# Piece values
PIECE_VALUES = {
    'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9,
    'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9,
}

# Pygame colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BOARD_SIZE = 600  # Increase the board size
SQUARE_SIZE = BOARD_SIZE // 8
INPUT_HEIGHT = 50  # Height of the input field

def print_board(board):
    unicode_board = str(board)

    lines = unicode_board.split('\n')
    for i in range(8):
        line = lines[i]
        for piece, unicode_piece in UNICODE_PIECES.items():
            line = line.replace(piece, unicode_piece)
        print(line)

def evaluate_board(board):
    total_value = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            total_value += PIECE_VALUES.get(piece.symbol(), 0)
    return total_value

def minimax(board, depth, maximizing_player):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    if maximizing_player:
        max_eval = -math.inf
        for move in generate_moves(board):
            board.push_uci(move)
            eval = minimax(board, depth - 1, False)
            board.pop()
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = math.inf
        for move in generate_moves(board):
            board.push_uci(move)
            eval = minimax(board, depth - 1, True)
            board.pop()
            min_eval = min(min_eval, eval)
        return min_eval

def best_move(board, depth):
    best_eval = -math.inf
    best_move = None
    for move in generate_moves(board):
        board.push_uci(move)
        eval = minimax(board, depth - 1, False)
        board.pop()
        if eval > best_eval:
            best_eval = eval
            best_move = move
    return best_move

def generate_moves(board):
    legal_moves = list(board.legal_moves)
    return [move.uci() for move in legal_moves]

def draw_board(screen, board):
    # Draw outer border
    pygame.draw.rect(screen, BLACK, (0, 0, BOARD_SIZE, BOARD_SIZE + INPUT_HEIGHT), 2)

    for row in range(8):
        for col in range(8):
            color = WHITE if (row + col) % 2 == 0 else BLACK
            pygame.draw.rect(screen, color, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
            piece = board.piece_at(chess.square(col, 7 - row))
            if piece is not None:
                piece_color = "white" if piece.color == chess.WHITE else "black"
                piece_type = get_piece_type(piece)
                piece_image = pygame.image.load(
                    f'C:/Users/faisa/Documents/Games_Section/chess/chess_asstes/{piece_color}_{piece_type}.png'
                )
                piece_image = pygame.transform.scale(piece_image, (SQUARE_SIZE, SQUARE_SIZE))
                screen.blit(piece_image, (col * SQUARE_SIZE, row * SQUARE_SIZE))

    # Draw semi-transparent vertical numbers
    for i in range(8):
        font = pygame.font.Font(None, 36)
        text = font.render(str(8 - i), True, BLACK)
        text.set_alpha(128)  # Set opacity to 128 (0.5)
        text_rect = text.get_rect()
        text_rect.center = (SQUARE_SIZE // 2, i * SQUARE_SIZE + SQUARE_SIZE // 2)
        screen.blit(text, text_rect)

    # Draw semi-transparent horizontal letters
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    for i in range(8):
        font = pygame.font.Font(None, 36)
        text = font.render(letters[i], True, BLACK)
        text.set_alpha(128)  # Set opacity to 128 (0.5)
        text_rect = text.get_rect()
        text_rect.center = (i * SQUARE_SIZE + SQUARE_SIZE // 2, BOARD_SIZE - SQUARE_SIZE // 2)
        screen.blit(text, text_rect)

def get_piece_type(piece):
    piece_type = piece.piece_type
    piece_names = {
        chess.PAWN: 'pawn',
        chess.ROOK: 'rook',
        chess.KNIGHT: 'knight',
        chess.BISHOP: 'bishop',
        chess.QUEEN: 'queen',
        chess.KING: 'king'
    }
    return piece_names.get(piece_type, 'unknown')

def main():
    pygame.init()
    screen = pygame.display.set_mode((BOARD_SIZE, BOARD_SIZE + INPUT_HEIGHT))
    pygame.display.set_caption("Chess Game")

    board = chess.Board()
    clock = pygame.time.Clock()

    input_text = ""
    font = pygame.font.Font(None, 36)

    while not board.is_game_over():
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                return
            elif event.type == KEYDOWN:
                if event.key == K_RETURN:
                    try:
                        move = chess.Move.from_uci(input_text)
                        if move.uci() in generate_moves(board):
                            board.push(move)
                            input_text = ""
                            evaluation = evaluate_board(board)
                            print(f"Evaluation after move: {evaluation}")
                        else:
                            print("Invalid move. Try again.")
                    except ValueError:
                        print("Invalid input. Enter moves in the format 'e2 e4'.")
                    continue
                elif event.key == K_BACKSPACE:
                    input_text = input_text[:-1]
                else:
                    input_text += event.unicode

        draw_board(screen, board)
        pygame.draw.rect(screen, WHITE, (0, BOARD_SIZE, BOARD_SIZE, INPUT_HEIGHT))
        pygame.draw.rect(screen, BLACK, (0, BOARD_SIZE, BOARD_SIZE, INPUT_HEIGHT), 2)
        input_surface = font.render(input_text, True, BLACK)
        input_rect = input_surface.get_rect()
        input_rect.midleft = (10, BOARD_SIZE + INPUT_HEIGHT // 2)
        screen.blit(input_surface, input_rect)

        pygame.display.flip()
        clock.tick(60)

        if not board.is_game_over() and not board.turn:
            ai_depth = 3
            ai_move_uci = best_move(board, ai_depth)
            if ai_move_uci:
                board.push_uci(ai_move_uci)
                evaluation = evaluate_board(board)
                print(f"AI's move: {ai_move_uci}")
                print(f"Evaluation after AI's move: {evaluation}")

    print("Game over!")

if __name__ == "__main__":
    main()



Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.
Invalid input. Enter moves in the format 'e2 e4'.


KeyboardInterrupt: 